<a href="https://colab.research.google.com/github/PariSsy/datacamp_notes/blob/main/Python_Writing_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Writing Functions in Python

* Instructor = Shayne Miel
* [Course Link](https://learn.datacamp.com/courses/writing-functions-in-python)
* Notes taken = Aug 9, 2021 by [Paris Zhang](https://www.linkedin.com/in/parisyunyuezhang/)



# Chapter 1 - Best Practices

## Section 1.1 - Docstrings

### A complex function

```
def split_and_stack( df, new_names ):
  """
  Split a DataFrame's columns into two halves and then stack them vertically, returning a new DataFrame with `new_names` as the column names.

  Args:
    df (DataFrame): The DataFrame to split.new_names (iterable of str): The column names for the new DataFrame.

  Returns:
    DataFrame
  """
  half = int(len(df.columns) / 2)
  left = df.iloc[:, :half]
  right = df.iloc[:, half:]
  return pd.DataFrame(
    data = np.vstack([left.values, right.values]),
    columns = new_names
  )
```

### Anatomy of a docstring

```
def function_name(arguments):
  """
  Description of what the function does.

  Description of the arguments, if any.

  Description of the return value(s), if any.

  Description of errors raised, if any.

  Optional extra notes or examples of usage.
  """
```

### Docstring formats
* Google Style
* Numpydoc
* reStructuredText
* EpyText

#### Google style - arguments & return value(s)

```
def function(arg_1, arg_2=42):
  """Description of what the function does.

  Args:
    arg_1 (str): Description of arg_1 that can break onto the next line if needed.
    arg_2 (int, optional): Write optional when an argument has a default value.
  
  Returns:
    bool: Optional description of the return value
    Extra lines are not indented.
  
  Raises:
    ValueError: Include any error types that the function intentionally raises.
  
  Notes:
    See https://www.datacamp.com/community/tutorials/docstrings-python for more info.
```

#### Numpydoc

```
def function(arg_1, arg_2=42):
  """
  Description of what the function does.

  Parameters
  ----------
  arg_1 : expected type of arg_1
    Description of arg_1.
  arg_2 : int, optional
    Write optional when an argument has a default value.
    Default = 42.
  
  Returns
  -------
  The type of the return value
    Can include a description of the return value.
    Replace "Returns" with "Yields" if this function is a generator.
  """
```

### Retrieving docstrings

```
def the_answer():
  """Return the answer to life, the universe, and everything.

  Returns:
    int
  """
  return 42
print(the_answer.__doc__)

>>> Return the answer to life, the universe, and everything.
>>>
>>>  Returns:
>>>    int
```

```
import insepct
print(inspect.getdoc(the_answer))

>>> Return the answer to life, the universe, and everything.
>>>
>>>  Returns:
>>>    int
```

### Examples

1. Count the number of times the letter appears in the string:

```
def count_letter(content, letter):
  """Count the number of times `letter` appears in `content`.

  Args:
    content (str): The string to search.
    letter (str): The letter to search for.

  Returns:
    int

  # Add a section detailing what errors might be raised
  Raises:
    ValueError: If `letter` is not a one-character string.
  """
  if (not isinstance(letter, str)) or len(letter) != 1:
    raise ValueError('`letter` must be a single character string.')
  return len([char for char in content if char == letter])
```

2. A feature that displays a tooltip with a function's docstring whenever the user starts typing the function name.

```
import inspect

def build_tooltip(function):
  """Create a tooltip for any function that shows the
  function's docstring.

  Args:
    function (callable): The function we want a tooltip for.

  Returns:
    str
  """
  # Get the docstring for the "function" argument by using inspect
  docstring = inspect.getdoc(function)
  border = '#' * 28
  return '{}\n{}\n{}'.format(border, docstring, border)

print(build_tooltip(count_letter))
print(build_tooltip(range))
print(build_tooltip(print))
```

## Section 1.2 - DRY and "Do One Thing"

### Don't repeat yourself (DRY)

* Training set
```
train = pd.read_csv('train.csv')
train_y = train['labels'].values
train_X = train[col for col in train.columns if col != 'labels'].values
train_pca = PCA(n_components=2).fit_transform(train_X)
plt.scatter(train_pca[:,0], train_pca[:,1])
```
* Validation set
```
val = pd.read_csv('validation.csv')
val_y = val['labels'].values
val_X = val[col for col in val.columns if col != 'labels'].values
val_pca = PCA(n_components=2).fit_transform(val_X)
plt.scatter(val_pca[:,0], val_pca[:,1])
```
* Testing set
```
test = pd.read_csv('test.csv')
test_y = test['labels'].values
test_X = test[col for col in test.columns if col != 'labels'].values
test_pca = PCA(n_components=2).fit_transform(test_X)
plt.scatter(test_pca[:,0], test_pca[:,1])
```

### Use functions to avoid repetition

```
def load_and_plot(path):
  """Load a data set and plot the first two principal components.

  Args:
    path (str): The location of a CSV file.
  
  Returns:
    tuple of ndarray: (features, labels)
  """
  data = pd.read_csv(path)
  y = data['label'].values
  X = data[col for col in train.columns if col != 'label'].values
  pca = PCA(n_components=2).fit_transform(X)
  plt.scatter(pca[:,0], pca[:,1])
  return X,y

train_X, train_y = load_and_plot('train.csv')
val_X, val_y = load_and_plot('val.csv')
test_X, test_y = load_and_plot('test.csv')
```

### Do One Thing

```
def load_data(path):
  """Load a data set.

  Args:
    path (str): The location of a CSV file.

  Returns:
    tuple of ndarray: (features, labels)
  """
  data = pd.read_csv(path)
  y = data['labels'].values
  X = data[col for col in data.columns if col != 'labels'].values
  return X, y
```

```
def plot_data():
  """Plot the first two principal components of a matrix.

  Args:
    X (numpy.ndarray): The data to plot.
  """
  pca = PCA(n_components=2).fit_transform(X)
  plt.scatter(pca[:,0], pca[:,1])
```

**Advantages**:
1. More flexible
2. More easily understood
3. Simpler to test
4. Simpler to debug
5. Easier to change

## Section 1.3 - Pass by assignment

### Example 1

```
def foo(x):
  x[0] = 99
my_list = [1,2,3]
foo(my_list)
print(my_list)

>>> [99,2,3]

def bar(x):
  x = x + 90
my_var = 3
bar(my_var)
print(my_var)

>>> 3
```

### Example 2

```
a = [1, 2, 3]
b = a
a.append(4)
print(b)

>>> [1, 2, 3, 4]

b.append(5)
print(a)

>>> [1, 2, 3, 4, 5]
```

### Immutable
* int
* float
* bool
* string
* bytes
* tuple
* frozenset
* None

### Mutable
* list
* dict
* set
* bytearray
* objects
* functions
* almost everything else

### Example where mutable default arguments are dangerous

```
def foo(var=[]):
  var.append(1)
  return var
foo()

>>> [1]

foo()

>>> [1, 1]
```

```
def foo(var=None):
  if var is None:
    var = []
  var.append(1)
  return var
foo()

>>> [1]

foo()

>>> [1]
```

### Exercise 1

```
def store_lower(_dict, _string):
  """Add a mapping between `_string` and a lowercased version of `_string` to `_dict`

  Args:
    _dict (dict): The dictionary to update.
    _string (str): The string to add.
  """
  orig_string = _string
  _string = _string.lower()
  _dict[orig_string] = _string

d = {}
s = 'Hello'

store_lower(d, s)

>>> d = {'Hello': 'hello'}
>>> s = 'Hello'
```

### Exercise 2

```
# Use an immutable variable for the default argument
def better_add_column(values, df=None):
  """Add a column of `values` to a DataFrame `df`.
  The column will be named "col_<n>" where "n" is
  the numerical index of the column.

  Args:
    values (iterable): The values of the new column
    df (DataFrame, optional): The DataFrame to update.
      If no DataFrame is passed, one is created by default.

  Returns:
    DataFrame
  """
  # Update the function to create a default DataFrame
  if df is None:
    df = pandas.DataFrame()
  df['col_{}'.format(len(df.columns))] = values
  return df
```

# Chapter 2 - Context Managers



## Section 2.1 - Using context managers

**Context manager structure**
* Sets up a context
* Runs code
* Removes the context

### A context manager example

Syntax:
```
with <context-manager>(<args>) as <variable-name>:
  # Run your code here
  # This code is running "inside the context"

# This code runs after the context is removed
```

Code:
```
with open('my_file.txt') as my_file:
  text = my_file.read()
  length = len(text)

print('The file is {} characters long'.format(length))
```

**`open()` does 3 things:**
* Sets up a context by opening a file
* Lets you run any code you want on that file
* Removes the context by closing the file

### Exercise 1 - the number of cats
Count the number of `cat(s)` in a text file:
```
# Open "alice.txt" and assign the file to "file"
with open('alice.txt') as file:
  text = file.read()

n = 0
for word in text.split():
  if word.lower() in ['cat', 'cats']:
    n += 1

print('Lewis Carroll uses the word "cat" {} times'.format(n))
```

### Exercise 2 - the speed of cats
Print runtime of each argument:
```
image = get_image_from_instagram()

# Time how long process_with_numpy(image) takes to run
with timer():
  print('Numpy version')
  process_with_numpy(image)

# Time how long process_with_pytorch(image) takes to run
with timer():
  print('Pytorch version')
  process_with_pytorch(image)
```

Output:
```
<script.py> output:
    Numpy version
    Processing..........done!
    Elapsed: 1.52 seconds
    Pytorch version
    Processing..........done!
    Elapsed: 0.33 seconds
```

## Section 2.2 - Writing context managers

Two ways to define a context manager:
1. Class-based
2. Function-based

### Syntax - create a context manager
1. Define a function.
2. (optional) Add any set up code your context needs.
3. Use the "yield" keyword.
4. (optional) Add any teardown code your context needs.
5. Add the `@contextlib.contextmanager` decorator.

```
@contextlib.contextmanager
def my_context():
  # Add any set up code you need
  yield
  # Add any teardown code you need
```

### The `yield` keyword
```
@contextlib.contextmanager
def my_context():
  print('hello')
  yield 42
  print('goodbye')
```

```
with my_context() as foo:
  print('foo is {}'.format(foo))
```
Output:
```
hello
foo is 42
goodbye
```

### Setup and teardown
```
@contextlib.contextmanager
def database(url):
  # set up database connection
  db = postgres.connect(url)
  
  yield db

  # tear down database connection
  db.disconnect()
```

```
url = 'http://datacamp.com/data'
with database(url) as my_db:
  course_list = my_db.execute(
    'SELECT * FROM courses'
  )
```

### Yielding a value or None
```
@contextlib.contextmanager
def in_dir(path):
  # save current working directory
  old_dir = os.getcwd()

  # switch to new working directory
  os.chdir(path)

  yield

  # change back to previous
  # working directory
  os.chdir(old_dir)
```

```
with in_dir('/data/project_1/'):
  project_files = os.listdir()
```

### Exercise 1 - the `timer()` context manager
```
# Add a decorator that will make timer() a context manager
@contextlib.contextmanager
def timer():
  """Time the execution of a context block.

  Yields:
    None
  """
  start = time.time()
  # Send control back to the context block
  yield None
  end = time.time()
  print('Elapsed: {:.2f}s'.format(end - start))

with timer():
  print('This should take approximately 0.25 seconds')
  time.sleep(0.25)
```

Output:
```
<script.py> output:
    This should take approximately 0.25 seconds
    Elapsed: 0.25s
```

### Exercise 2 - a read-only `open()` context manager
The regular `open()` context manager:
* takes a filename and a mode (`'r'` for read, `'w'` for write, or `'a'` for append)
* opens the file for reading, writing, or appending
* yields control back to the context, along with a reference to the file
* waits for the context to finish
* and then closes the file before exiting

```
@contextlib.contextmanager
def open_read_only(filename):
  """Open a file in read-only mode.

  Args:
    filename (str): The location of the file to read

  Yields:
    file object
  """
  read_only_file = open(filename, mode='r')
  # Yield read_only_file so it can be assigned to my_file
  yield read_only_file
  # Close read_only_file
  read_only_file.close()

with open_read_only('my_file.txt') as my_file:
  print(my_file.read())
```

## Section 2.3 - Advanced topics

### Nested contexts

Original syntax:
```
def copy(src, dst):
  """Copy the contents of one file to another.

  Args:
    src (str): File name of the file to be copied.
    dst (str): Where to write the new file.
  """

  # Open the source file and read in the contents
  with open(src) as f_src:
    contents = f_src.read()

  # Open the destination file and write out the contents
  with open(dst, 'w') as f_dst:
    f_dst.write(contents)
```

Nesting:
```
with open('my_file.txt') as my_file:
  for line in my_file:
    # do something
```

Nested syntax:
```
def copy(src, dst):
  """Copy the contents of one file to another.

  Args:
    src (str): File name of the file to be copied.
    dst (str): Where to write the new file.
  """

  # Open both files
  with open(src) as f_src:
    with open(dis, 'w') as f_dst:
      # Read and write each line, one at a time
      for line in f_src:
        f_dst.write(contents)
```

### Handling errors

Original syntax:
```
def get_printer(ip):
  p = connect_to_printer(ip)

  yield

  # This MUST be called or no one else will
  # be able to connect to the printer
  p.disconnect()
  print('disconnected from printer')

doc = {'text': 'This is my text.'}

with get_printer('10.0.34.111') as printer:
  printer.print_page(doc['txt'])
```

Error:
```
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
    printer.print_page(doc['txt'])
KeyError: 'txt'
```

Syntax for handling errors:
```
try:
  # code that might raise an error
except:
  # do something about the error
finally:
  # this code runs no matter what
```

Corrected syntax:
```
def get_printer(ip):
  p = connect_to_printer(ip)

  try:
    yield
  finally:
    p.disconnect()
    print('disconnected from printer')

doc = {'text': 'This is my text.'}

with get_printer('10.0.34.111') as printer:
  printer.print_page(doc['txt'])
```

Output:
```
disconnected from printer
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
    printer.print_page(doc['txt'])
KeyError: 'txt'
```

### Context manager patterns
1. Open - Close
2. Lock - Release
3. Change - Reset
4. Enter - Exit
5. Start - Stop
6. Setup - Teardown
7. Connect - Disconnect

Adapted from Dave Brondsema's talk at PyCon 2012 ([Reference](https://youtu.be/cSbD5SKwak0?t=795))

### Exercise 1 - scraping the NASDAQ
(Nested contexts)
```
# Use the "stock('NVDA')" context manager
# and assign the result to the variable "nvda"
with stock('NVDA') as nvda:
  # Open "NVDA.txt" for writing as f_out
  with open('NVDA.txt', 'w') as f_out:
    for _ in range(10):
      value = nvda.price()
      print('Logging ${:.2f} for NVDA'.format(value))
      f_out.write('{:.2f}\n'.format(value))
```

output:
```
<script.py> output:
    Opening stock ticker for NVDA
    Logging $139.50 for NVDA
    Logging $139.54 for NVDA
    Logging $139.61 for NVDA
    Logging $139.65 for NVDA
    Logging $139.72 for NVDA
    Logging $139.73 for NVDA
    Logging $139.80 for NVDA
    Logging $139.78 for NVDA
    Logging $139.73 for NVDA
    Logging $139.64 for NVDA
    Closing stock ticker
```

### Exercise 2 - changing the working directory
(Handling errors)
```
def in_dir(directory):
  """Change current working directory to `directory`,
  allow the user to run some code, and change back.

  Args:
    directory (str): The path to a directory to work in.
  """
  current_dir = os.getcwd()
  os.chdir(directory)

  # Add code that lets you handle errors
  try:
    yield
  # Ensure the directory is reset,
  # whether there was an error or not
  finally:
    os.chdir(current_dir)
```

# Chapter 3 - Decorators



## Section 3.1 - Functions are objects

### 1. Functions as variables
Input
```
def my_function():
  print('Hello')
x = my_function
type(x)

>>> <type 'function'>

x()

>>> Hello

PrintyMcPrintface = print
PrintyMcPrintface('Python is awesome!')

>>> Python is awesome!
```

### 2. Lists and dictionaries of functions
```
list_of_functions = [my_function, open, print]
list_of_functions[2]('I am printing with an element of a list!')

>>> I am printing with an element of a list!

dict_of_functions = {
'func1': my_function,
'func2': open,
'func3': print
}
dict_of_functions['func3']('I am printing with a value of a dict!')

>>> I am printing with a value of a dict!
```

### 3. Referencing a function
```
def my_function():
  return 42

my_function()

>>> 42

my_function

>>> <function my_function at 0x7f475332a730>
```

### 4. Functions as arguments
```
def has_docstring(func):
  """Check to see if the function
  `func` has a docstring.
  
  Args:
    func (callable): A function.
  
  Returns:
    bool
  """
  return func.__doc__is not None

def no():
  return 42

def yes():
  """Return the value 42
  """
  return 42

has_docstring(no)

>>> False

has_docstring(yes)

>>> True
```

### 5. Defining a function inside another function
```
def foo():
  x = [3, 6, 9]

  def bar(y):
    print(y)

  for value in x:
    bar(x)
```

```
def foo(x, y):
  if x > 4 and x < 10 and y > 4 and y < 10:
    print(x * y)
```

```
def foo(x, y):
  def in_range(v):
    return v > 4 and v < 10

  if in_range(x) and in_range(y):
    print(x * y)
```

### 6. Functions as return values
```
def get_function():
  def print_me(s):
    print(s)
    
  return print_me

new_func = get_function()
new_func('This is a sentence.')

>>> This is a sentence.
```

### Exercise 1
```
# Add the missing function references to the function map
function_map = {
  'mean': mean,
  'std': std,
  'minimum': minimum,
  'maximum': maximum
}

data = load_data()
print(data)

func_name = get_user_input()

# Call the chosen function and pass "data" as an argument
function_map[func_name](data)
```

### Exercise 2
```
def create_math_function(func_name):
  if func_name == 'add':
    def add(a, b):
      return a + b
    return add
  elif func_name == 'subtract':
    # Define the subtract() function
    def subtract(a, b):
      return a - b
    return subtract
  else:
    print("I don't know that one")
    
add = create_math_function('add')
print('5 + 2 = {}'.format(add(5, 2)))

subtract = create_math_function('subtract')
print('5 - 2 = {}'.format(subtract(5, 2)))

<script.py> output:
    5 + 2 = 7
    5 - 2 = 3
```

## Section 3.2 - Scope
1. Builtin
2. Global
3. Nonlocal
4. Local

### The `global` keyword
Local
```
x = 7

def foo():
  x = 42
  print(x)

foo()
>>> 42

print(x)
>>> 7
```

Global
```
x = 7

def foo():
  global x
  x = 42
  print(x)

foo()
>>> 42

print(x)
>>> 42
```

### The `nonlocal` keyword

Local
```
def foo():
  x = 10

  def bar():
    x = 200
    print(x)
  
  bar()
  print(x)

foo()
>>> 200
>>> 10
```

Nonlocal
```
def foo():
  x = 10

  def bar():
    nonlocal x
    x = 200
    print(x)
  
  bar()
  print(x)

foo()
>>> 200
>>> 200
```

### Exercise 1
```
call_count = 0

def my_function():
  # Use a keyword that lets us update call_count 
  global call_count
  call_count += 1
  
  print("You've called my_function() {} times!".format(
    call_count
  ))
  
for _ in range(20):
  my_function()

<script.py> output:
    You've called my_function() 1 times!
    You've called my_function() 2 times!
    ...
    You've called my_function() 20 times!
```

### Exercise 2
```
def read_files():
  file_contents = None
  
  def save_contents(filename):
    # Add a keyword that lets us modify file_contents
    nonlocal file_contents
    if file_contents is None:
      file_contents = []
    with open(filename) as fin:
      file_contents.append(fin.read())
      
  for filename in ['1984.txt', 'MobyDick.txt', 'CatsEye.txt']:
    save_contents(filename)
    
  return file_contents

print('\n'.join(read_files()))

<script.py> output:
    It was a bright day in April, and the clocks were striking thirteen.
    Call me Ishmael.
    Time is not a line but a dimension, like the dimensions of space.
```

### Exercise 3
```
def wait_until_done():
  def check_is_done():
    # Add a keyword so that wait_until_done() 
    # doesn't run forever
    global done
    if random.random() < 0.1:
      done = True
      
  while not done:
    check_is_done()

done = False
wait_until_done()

print('Work done? {}'.format(done))

<script.py> output:
    Work done? True
```

## Section 3.3 - Closures

### Attaching nonlocal variables to nested functions
```
def foo():
  a = 5
  def bar():
    print(a)
  return bar

func = foo()

func()
>>> 5
```

Closures
```
type(func.__closure__)
>>> <class 'tuple'>

len(func.__closure__)
>>> 1

func.__closure__[0].cell_contents
>>> 5
```

### Closures and deletion
```
x = 25

def foo():
  def bar():
    print(a)
  return bar

my_func = foo()
my_func()
>>> 25
```

Deletion
```
del(x)
my_func()
>>> 25

len(my_func.__closure__)
>>> 1

my_func.__closure__[0].cell_contents
>>> 25
```

### Closures and overwriting
```
x = 25

def foo():
  def bar():
    print(a)
  return bar

x = foo()
x()
>>> 25
```

Overwriting
```
len(x.__closure__)
>>> 1

x.__closure__[0].cell_contents
>>> 25
```

### Definitions

#### Nested function
A function defined inside another function
```
# outer function
def parent():
  # nested function
  def child():
    pass
  return child
```

#### Nonlocal variabls
Variables defined in the parent function that are used by the child funtion.
```
def parent(arg_1, arg_2):
  # From child()'s point of view,
  # `value` and `my_dict` are nonlocal variables,
  # as are `arg_1` and `arg_2`.
  value = 22
  my_dict = {'chocolate': 'yummy'}

  def child():
    print(2 * value)
    print(my_dict['chocolate'])
    print(arg_1 + arg_2)
  
  return child
```

#### Closure
Nonlocal variables attached to a returned function.
```
def parent(arg_1, arg_2):
  # From child()'s point of view,
  # `value` and `my_dict` are nonlocal variables,
  # as are `arg_1` and `arg_2`.
  value = 22
  my_dict = {'chocolate': 'yummy'}

  def child():
    print(2 * value)
    print(my_dict['chocolate'])
    print(arg_1 + arg_2)
  
  return child

new_function = parent(3, 4)

print([cell.cell_contents for cell in new_function.__closure__])

>>> [3, 4, 22, {'chocolate': 'yummy'}]
```

### Why matters?
Decorators use:
* Functions as objects
* Nested functions
* Nonlocal scope
* Closures

### Exercise 1
```
def return_a_func(arg1, arg2):
  def new_func():
    print('arg1 was {}'.format(arg1))
    print('arg2 was {}'.format(arg2))
  return new_func
    
my_func = return_a_func(2, 17)

print(my_func.__closure__ is not None)
print(len(my_func.__closure__) == 2)

closure_values = [
  my_func.__closure__[i].cell_contents for i in range(2)
]
print(closure_values == [2, 17])

<script.py> output:
    True
    True
    True
```

### Comments
1. You can modify, delete, or overwrite the values needed by the nested function, the the nested function can still those values because they are stored safely in the function's closure.
2. You could run into memory issues if you wound up adding a very large array or object to the closure.

## Section 3.4 - Decorators

### The double_args decorator
```
def double_args(func):
  def wrapper(a, b):
    return func(a * 2, b * 2)
  return wrapper

def multiply(a, b):
  return a * b

multiply = double_args(multiply)

multiply(1, 5)
>>> 20
```

### Decorator syntax
```
def double_args(func):
  def wrapper(a, b):
    return func(a * 2, b * 2)
  return wrapper

@double_args
def multiply(a, b):
  return a * b

multiply(1, 5)
>>> 20
```

### Exercise 2
```
def print_before_and_after(func):
  def wrapper(*args):
    print('Before {}'.format(func.__name__))
    # Call the function being decorated with *args
    func(*args)
    print('After {}'.format(func.__name__))
  # Return the nested function
  return wrapper

@print_before_and_after
def multiply(a, b):
  print(a * b)

multiply(5, 10)

<script.py> output:
    Before multiply
    50
    After multiply
```

# Chapter 4 - More on Decorators


## Section 4.1 - Examples

### Time a function

Pseudo code
```
import time

def timer(func):
  """A decorator that prints how long a function took to run.

  Args:
    func (callable): The function being decorated.

  Returns:
    callable: The decorated function.
  """
```

Define timer()
```
import time

def timer(func):
  """A decorator that prints how long a function took to run."""
  # Define the wrapper function to return.
  def wrapper(*args, **kwargs):
    # When wrapper() is called, get the current time.
    t_start = time.time()
    # Call the decorated function and store the result.
    result = func(*args, **kwargs)
    # Get the total time it took to run, and print it.
    t_total = time.time() - t_start
    print('{} took {}s'.format(func.__name__, t_total))
    return result
  return wrapper
```

Use timer()
```
@timer
def sleep_n_seconds(n):
  time.sleep(n)

sleep_n_seconds(5)
>>> sleep_n_seconds took 5.0050950050354s

sleep_n_seconds(10)
>>> sleep_n_seconds took 10.010067701339722s
```

### Memorize the result

Define memorize()
```
def memorize(func):
  """Store the results of the decorated function for fast lookup
  """
  # Store results in a dict that maps arguments to results
  cache = {}
  # Define the wrapper function to return.
  def wrapper(*args, **kwargs):
    # If these arguments haven't been seen before,
    if (args, kwargs) not in cache:
      # Call func() and store the result.
      cache[(args, kwargs)] = func(*args, **kwargs)
    return cache[(args, kwargs)]
  return wrapper
```

Use memorize()
```
@memorize
def slow_function(a, b):
  print('Sleeping...')
  time.sleep(5)
  return a + b

slow_function(3, 4)
>>> Sleeping...
>>> 7

slow_function(3, 4)
>>> 7
```

### When to use decorators?
* Add common behavior to multiple functions

```
@timer
def foo():
  # do some computation

@timer
def bar():
  # do some other computation

@timer
def baz():
  # do something else
```

### Exercise 1
Decorate a function to print its return type.
```
def print_return_type(func):
  # Define wrapper(), the decorated function
  def wrapper(*args, **kwargs):
    # Call the function being decorated
    result = func(*args, **kwargs)
    print('{}() returned type {}'.format(
      func.__name__, type(result)
    ))
    return result
  # Return the decorated function
  return wrapper
  
@print_return_type
def foo(value):
  return value
  
print(foo(42))
print(foo([1, 2, 3]))
print(foo({'a': 42}))

<script.py> output:
    foo() returned type <class 'int'>
    42
    foo() returned type <class 'list'>
    [1, 2, 3]
    foo() returned type <class 'dict'>
    {'a': 42}
```

### Exercise 2
Decorate a function to count how many times it was called.
```
def counter(func):
  def wrapper(*args, **kwargs):
    wrapper.count += 1
    # Call the function being decorated and return the result
    return wrapper.count
  wrapper.count = 0
  # Return the new decorated function
  return wrapper

# Decorate foo() with the counter() decorator
@counter
def foo():
  print('calling foo()')
  
foo()
foo()

print('foo() was called {} times.'.format(foo.count))

<script.py> output:
    foo() was called 2 times.
```

## Section 4.2 - Decorators and metadata (Preserving docstrings and more)

```
def sleep_n_seconds(n=10):
  """Pause processing for n seconds.

  Args:
    n (int): The number of seconds to pause for.
  """
  time.sleep(n)
```

Check contents
```
print(sleep_n_seconds.__doc__)
>>> Pause processing for n seconds
>>>
>>>  Args:
>>>    n (int): The number of seconds to pause for.

print(sleep_n_seconds.__name__)
>>> sleep_n_seconds

print(sleep_n_seconds.__defaults__)
>>> (10,)
```

Add timer()
```
@timer
def sleep_n_seconds(n=10):
  """Pause processing for n seconds.

  Args:
    n (int): The number of seconds to pause for.
  """
  time.sleep(n)
```

Check contents
```
print(sleep_n_seconds.__doc__)
>>>

print(sleep_n_seconds.__name__)
>>> wrapper
```

### Access the original function

Review the definition
```
def timer(func):
  """A decorator that prints how long a function took to run."""
 
  def wrapper(*args, **kwargs):
    t_start = time.time()

    result = func(*args, **kwargs)

    t_total = time.time() - t_start
    print('{} took {}s'.format(func.__name__, t_total))

    return result

  return wrapper
```

Modify timer()
```
from functools import wraps
def timer(func):
  """A decorator that prints how long a function took to run."""
 
  @wraps(func)
  def wrapper(*args, **kwargs):
    t_start = time.time()

    result = func(*args, **kwargs)

    t_total = time.time() - t_start
    print('{} took {}s'.format(func.__name__, t_total))

    return result

  return wrapper
```

Add timer()
```
@timer
def sleep_n_seconds(n=10):
  """Pause processing for n seconds.

  Args:
    n (int): The number of seconds to pause for.
  """
  time.sleep(n)
```

Check contents
```
print(sleep_n_seconds.__doc__)
>>> Pause processing for n seconds
>>>
>>>  Args:
>>>    n (int): The number of seconds to pause for.

print(sleep_n_seconds.__name__)
>>> sleep_n_seconds

print(sleep_n_seconds.__defaults__)
>>> (10,)

sleep_n_seconds.__wrapped__
>>> <function sleep_n_seconds at 0x7f52cab44ae8>
```

### Exercise 1
Return the original docstrings of a decorated function.
```
from functools import wraps

def add_hello(func):
  # Decorate wrapper() so that it keeps func()'s metadata
  @wraps(func)
  def wrapper(*args, **kwargs):
    """Print 'hello' and then call the decorated function."""
    print('Hello')
    return func(*args, **kwargs)
  return wrapper
  
@add_hello
def print_sum(a, b):
  """Adds two numbers and prints the sum"""
  print(a + b)
  
print_sum(10, 20)
print_sum_docstring = print_sum.__doc__
print(print_sum_docstring)

<script.py> output:
    Hello
    30
    Adds two numbers and prints the sum
```

### Exercise 2
Compare runtime with and without a decorator.
```
@check_everything
def duplicate(my_list):
  """Return a new list that repeats the input twice"""
  return my_list + my_list

t_start = time.time()
duplicated_list = duplicate(list(range(50)))
t_end = time.time()
decorated_time = t_end - t_start

t_start = time.time()
# Call the original function instead of the decorated one
duplicated_list = duplicate.__wrapped__(list(range(50)))
t_end = time.time()
undecorated_time = t_end - t_start

print('Decorated time: {:.5f}s'.format(decorated_time))
print('Undecorated time: {:.5f}s'.format(undecorated_time))

<script.py> output:
    Finished checking inputs
    Finished checking outputs
    Decorated time: 1.51280s
    Undecorated time: 0.00006s
```

## Section 4.3 - Decorators that take arguments

## Section 4.4 - Timeout() example